In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir drive/MyDrive/DATA_MINING_COURSEWORK/TAB_FACT_EMBEDDINGS/table_text

In [ ]:
%pip install datasets
%pip install numpy
%pip install pandas
%pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import torch

In [ ]:
dataset = load_dataset("tab_fact", "tab_fact")

Generating train split:   0%|          | 0/92283 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/12792 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12779 [00:00<?, ? examples/s]

In [ ]:
df = dataset["train"].to_pandas()

In [ ]:
df.isna().sum()

id               0
table_id         0
table_text       0
table_caption    0
statement        0
label            0
dtype: int64

In [ ]:
X1_split, X2_split = np.split(df["table_text"], 1619), np.split(df["statement"], 1619) # exact divisor, gives 57 splits

Y_split = np.split(df["label"], 1619)

In [ ]:
from transformers import BertTokenizer, BertModel

model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name).to("cuda")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:

for split in range(len(X1_split)):
    embeddings_x1 = []
    embeddings_x2 = []
    print (f"Generating embeddings split: {split}")
    for table_text, statement in zip(X1_split[split], X2_split[split]):
        input_x1 = tokenizer(table_text, return_tensors="pt", padding=True, truncation=True).to("cuda")
        input_x2 = tokenizer(statement, return_tensors="pt", padding=True, truncation=True).to("cuda")
        output_x1 = model(**input_x1)
        output_x2 = model(**input_x2)
        cls_embedding_x1 = output_x1.last_hidden_state.mean(dim=1)
        cls_embedding_x2 = output_x2.last_hidden_state.mean(dim=1)
        embeddings_x1.append(cls_embedding_x1.detach().cpu().numpy())
        embeddings_x2.append(cls_embedding_x2.detach().cpu().numpy())
    embeddings_x1 = np.array(embeddings_x1)
    embeddings_x2 = np.array(embeddings_x2)
    embeddings_x1 = np.array(embeddings_x1).reshape(-1, embeddings_x1.shape[-1])
    embeddings_x2 = np.array(embeddings_x2).reshape(-1, embeddings_x2.shape[-1])
    np.save(f"drive/MyDrive/DATA_MINING_COURSEWORK/TAB_FACT_EMBEDDINGS/table_text/embeddings_table_text_{split}", embeddings_x1)
    np.save(f"drive/MyDrive/DATA_MINING_COURSEWORK/TAB_FACT_EMBEDDINGS/statement/embeddings_statement_{split}", embeddings_x2)

Generating embeddings split: 0
Generating embeddings split: 1
Generating embeddings split: 2
Generating embeddings split: 3
Generating embeddings split: 4
Generating embeddings split: 5
Generating embeddings split: 6
Generating embeddings split: 7
Generating embeddings split: 8
Generating embeddings split: 9
Generating embeddings split: 10
Generating embeddings split: 11
Generating embeddings split: 12
Generating embeddings split: 13
Generating embeddings split: 14
Generating embeddings split: 15
Generating embeddings split: 16
Generating embeddings split: 17
Generating embeddings split: 18
Generating embeddings split: 19
Generating embeddings split: 20
Generating embeddings split: 21
Generating embeddings split: 22
Generating embeddings split: 23
Generating embeddings split: 24
Generating embeddings split: 25
Generating embeddings split: 26
Generating embeddings split: 27
Generating embeddings split: 28
Generating embeddings split: 29
Generating embeddings split: 30
Generating embeddi